## Retrieving Collections

https://docs.opensea.io/reference/retrieving-collections

Harrison is investigating as of 2/23

### Initial data gathering request

Here is the link to the user we are testing this method with:
https://opensea.io/ARTSNFT

In [1]:
# Create initial search, manual for now
# Only doing this because I'm not sure if we currently save this info
username = 'ARTSNFT'
owner_url = 'opensea.io/ARTSNFT'
asset_owner_wallet_url = '0xf51e77f49c8a31174c96462dd6235cdc4df9fbc4'

In [2]:
import requests

url = "https://api.opensea.io/api/v1/collections?asset_owner={}&offset=0&limit=300".format(asset_owner_wallet_url)

headers = {"Accept": "application/json"}

response = requests.request("GET", url, headers=headers)

### Initial formatting

In [3]:
import json
import pandas as pd
import numpy as np

In [4]:
response_text = json.loads(response.text)
df = pd.json_normalize(response_text)

The dataframe looks good overall, but the 'primary_asset_contracts' column is messy. It is a column of lists with a length of 1. The only element within that list is a dictionary.

In [5]:
df.head(2)

,primary_asset_contracts,banner_image_url,chat_url,created_date,default_to_fiat,description,dev_buyer_fee_basis_points,dev_seller_fee_basis_points,discord_url,external_url,...,stats.total_sales,stats.total_supply,stats.count,stats.num_owners,stats.average_price,stats.num_reports,stats.market_cap,stats.floor_price,display_data.card_display_style,display_data.images
0,[{'address': '0x9d8826881a2beab386a7858e59c228...,https://lh3.googleusercontent.com/-mtSJ6aFZGP_...,None,2022-02-21T15:37:33.224465,False,Gen 0 Bloodshed Bears are the first Bloodshed ...,0,750,None,https://bloodshedbears.com/,...,3751.0,7000.0,7000.0,2392,0.260240,30,1821.678312,0,contain,NaN
1,[{'address': '0xb4d06d46a8285f4ec79fd294f78a88...,https://lh3.googleusercontent.com/nzpEHnEfOg65...,None,2022-02-19T13:01:18.111966,False,3Landers is a collectible NFT project centered...,0,650,https://discord.gg/3landers,https://3landersnft.com/,...,9699.0,10000.0,10000.0,5102,1.383236,7,13832.359135,0,cover,NaN


In [6]:
print("Each element in the series is", type(df['primary_asset_contracts'][0]))
print("Each of these lists has a length of", len(df['primary_asset_contracts'][0]))
print("The only element within these lists is of type", type(df['primary_asset_contracts'][0][0]))

Each element in the series is <class 'list'>
Each of these lists has a length of 1
The only element within these lists is of type <class 'dict'>


The next step is to convert this column of dictionaries (each within their own list) into a set of columns, which will be re-combined with the original dataframe. The original column, "primary_asset_contracts", will be removed from the dataframe.

The newly-created columns, which expand on the original column ("primary_asset_contracts") will all start with "pac_" before continuing their original names. The original names were the keys in the dictionaries.

In [7]:
# Create the initial dataframe from the first row
pac_df = pd.DataFrame(df.iloc[0]['primary_asset_contracts'][0], index=[0])

# Loop through the rest
for pac in df['primary_asset_contracts'][1:]:
    try:
        temp_df = pd.DataFrame(pac[0], index=[0])
# The IndexError comes from an empty list
# Solution: append an empty df with same column names
    except IndexError:
        empty_d = {}
        for col in pac_df.columns:
            empty_d[col] = np.nan
        temp_df = pd.DataFrame(empty_d, index=[0])
    pac_df = pac_df.append(temp_df)
    
# Adding "pac_" to start of each column name to track them later
pac_df.rename(columns=lambda x: "pac_"+x, inplace=True)
# All rows currently have index of 0, dropping and replacing with standard range indexes
pac_df = pac_df.reset_index(drop=True)

In [8]:
# Recombine the dataframes
df = df.join(pac_df)
# Drop the original column
df = df.drop(columns=['primary_asset_contracts'])

In [9]:
# Showing the columns just because I can't see them all with df.head()
df.columns

Index(['banner_image_url', 'chat_url', 'created_date', 'default_to_fiat',
       'description', 'dev_buyer_fee_basis_points',
       'dev_seller_fee_basis_points', 'discord_url', 'external_url',
       'featured', 'featured_image_url', 'hidden', 'safelist_request_status',
       'image_url', 'is_subject_to_whitelist', 'large_image_url',
       'medium_username', 'name', 'only_proxied_transfers',
       'opensea_buyer_fee_basis_points', 'opensea_seller_fee_basis_points',
       'payout_address', 'require_email', 'short_description', 'slug',
       'telegram_url', 'twitter_username', 'instagram_username', 'wiki_url',
       'owned_asset_count', 'stats.one_day_volume', 'stats.one_day_change',
       'stats.one_day_sales', 'stats.one_day_average_price',
       'stats.seven_day_volume', 'stats.seven_day_change',
       'stats.seven_day_sales', 'stats.seven_day_average_price',
       'stats.thirty_day_volume', 'stats.thirty_day_change',
       'stats.thirty_day_sales', 'stats.thirty_day_aver

## Adding contextual information

In [10]:
df['owner_user'] = username
df['owner_url'] = owner_url
df['asset_owner_wallet_url'] = asset_owner_wallet_url

In [11]:
# Exporting to csv
df.to_csv('initial_opensea_get_collections.csv')